## Figures for Network model of Latent Recurrent Discharge (LRD)

### Some imports and settings

In [1]:
import brian2.numpy_ as np

from settings import logging, time_unit

logger = logging.getLogger("lrd")

plot_fig = True
file_formats = ["pdf", "png"]
save_args = dict(use_args=True, close=True, file_formats=["pdf"])


### Base Figure

In [2]:
from core.sim import single_run
net, results = single_run()

2023-04-16 12:15:32 [   DEBUG]                                                                                            (sim::sim.py::149)
2023-04-16 12:15:32 [   DEBUG] scale = 1.0000                                                                             (sim::sim.py::240)
2023-04-16 12:15:32 [   DEBUG] EGABA = -74.00                                                                             (sim::sim.py::278)
2023-04-16 12:15:32 [   DEBUG] static_cl_units = 14.0000mV                                                                (sim::sim.py::294)
2023-04-16 12:15:32 [   DEBUG] manual_cl                                                                                  (sim::sim.py::346)
2023-04-16 12:15:32 [   DEBUG] facilitating synapses                                                                      (sim::sim.py::401)
WARNING    Cannot check whether the indices to record from are valid. This can happen in standalone mode when recording from synapses that have been creat

Starting simulation at t=0 s for duration 90 s
22.8384 s (25%) simulated in 10s, estimated 29s remaining.
45.9423 s (51%) simulated in 20s, estimated 19s remaining.
69.1494 s (76%) simulated in 30s, estimated 9s remaining.
90 s (100%) simulated in 39s


2023-04-16 12:16:37 [    INFO] took 64.11s to run                                                                         (sim::sim.py::668)


In [9]:
brian2plot

MagicNetwork()

## Explain figure

In [ ]:
from lrdfigure_explain import Explain

np.random.seed(1337)

save_args = dict(use_args=False, close=True, file_formats=["pdf"])

explain = Explain()
ehco3 = -18
phco3 = 0.2
pcl = 1 - phco3
mv_step = 2
time_per_value = 60
egaba = [-74, -40]
diff = egaba[1] - egaba[0]
values = diff // mv_step
steps = values - 1
duration = values * time_per_value
ecl = [round((e - phco3 * ehco3) / pcl, 2) for e in egaba]
# [-74, -40] -> 34 dif /2 -> 17 values -1 -> 16 steps -> 100 s per value -> 1600 s
explain.run(
    duration=duration,
    num_ecl_steps=steps,
    E_Cl_0=ecl[0],
    E_Cl_end=ecl[1],
)
explain.plot()
explain.save_figure(**save_args)
del explain


## Alter conductances

In [ ]:
from lrdfigure_drugs import Drugs

ehco3 = -18
phco3 = 0.2
pcl = 1 - phco3
egabas = [-74, -60, -58, -46]
E_Cl_0s = [round((e - phco3 * ehco3) / pcl, 2) for e in egabas]
drugs = Drugs(benzo_strengths=(0, 0.25, 0.5, 1, 2, 3, 4, 5), E_Cl_0s=E_Cl_0s)
drugs.run()
drugs.plot()
drugs.save_figure(**save_args)
del drugs

## Start having dynamic chloride $Cl^-$

In [ ]:
from lrdfigure_chloride import Chloride


cl = Chloride(seeds=(None, 1038, 1337, 1111, 1010, 1011, 1101, 1110, 11110, 111100))
cl.run(duration=600)
cl.plot(timeit=True, colorbar=False)
cl.save_figure(**save_args)
del cl


## Explore time constants of "Excitatory" and "Inhibitory" KCC2

$\tau_{KCC2_{exc}}$

$\tau_{KCC2_{inh}}$

In [ ]:
from lrdfigure_tau import Tau

tau_KCC2_E_list = [int(g) for g in np.geomspace(50, 800, 9).round(0)]
    # tau_KCC2_I_list = [50, 250, 375, 500, 800, 1000]
tau_KCC2_I_list = tau_KCC2_E_list
tau = Tau(
    tau_KCC2_E_list=tau_KCC2_E_list,
    tau_KCC2_I_list=tau_KCC2_I_list,
    g_GABA_list=[50, 250],
    seeds=(
        None,
        1038,
        1337,
        1111,
        1010,
    ),
)
tau.run(duration=600, nrn_idx_i=[0, 1, 2, 3]).plot(
    plot_g_GABA_list=[50, 250]
).save_figure(**save_args)

del tau


## $G_{GABA}$ vs $\tau_{KCC2}$ with $E_{GABA}$

In [ ]:
from lrdfigure_ggaba_e import Gve


gve = Gve(
    seeds=(None, 1234, 5678),
    gGABAsvEGABA=sorted(
        set(
            np.append(
                np.round(np.arange(0, 100.0001, 10), 0),
                np.geomspace(10, 1000, 11).round(0),
            )
        )
    ),
    gGABAs=np.geomspace(10, 1000, 11).round(0),
    tau_KCC2s=[int(g) for g in np.geomspace(50, 800, 9).round(0)],
)
gve.run()
gve.process()
gve.plot()
gve.save_figure(figs=gve.figs, close=True)


In [ ]:
gve = Gve(
    gGABAs=np.append(np.round(np.arange(0, 100.0001, 10), 0), [125, 160, 200, 250]),
    # gAMPAs=np.round(np.arange(0, 10.0001, 1), 0),
    tau_KCC2s=[int(g) for g in np.geomspace(200, 400, 9).round(0)],
)
gve.run()
gve.process()
gve.plot()
gve.sim_name = gve.fig_name + "_supp"
gve.save_figure(figs=gve.figs, use_args=True, close=True)


## Supplemenatry Work

In [ ]:
from lrdfigure_gampa import Params

gampa = Params(
    gGABAs=np.append(np.round(np.arange(0, 100.0001, 10), 0), [125, 160, 200, 250]),
    gAMPAs=np.round(np.arange(0, 10.0001, 1), 0),
)
gampa.run()
gampa.process()
gampa.plot()
gampa.save_figure(figs=gampa.figs, close=True)